In [ ]:
#import libraries for the project
import pandas as pd
import requests 
import json
from datetime import datetime, timezone
import sqlalchemy
from sqlalchemy import create_engine


In [ ]:
#The api_token to connect with exchangerate.host
api_token = "8ed177bf655d362a495251f500f37a35"

In [ ]:
#Extracting the currencies rate from exchangerate.host
def extract_curr(base="IDR"):
  url = f"https://api.exchangerate.host/live?access_key={api_token}&source={base}"
  response = requests.get(url)
  data = response.json()

  return data

In [ ]:
def transform_curr(data):
    rates = data["quotes"]

    #Transform the json to dataframes
    df = pd.DataFrame(rates.items(), columns = ["Target_Currency","Exchange_Rate"])

    #Adding and formatting tables for bettter data insight
    df["Target_Currency"] = df["Target_Currency"].str.replace("IDR","")
    df["Base_Currency"] = data["source"]
    df["Date"] = datetime.fromtimestamp(data["timestamp"], tz = timezone.utc).strftime('%Y-%m-%d')
    df["Rate_in_IDR"] = 1 / df["Exchange_Rate"]
    
    #Opening the currency_list.json
    with open("currency_list.json","r") as file:
        curr = json.load(file)
    currency_df = pd.DataFrame(curr["currencies"].items(), columns = ["Target_Currency","Description"])

    #Merge the dataframes by Target_Currency    
    df = df.merge(currency_df, on = "Target_Currency", how = "left")

    #Created the Created_at column to get the information when the data is extracted
    df["Created_at"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    #Format the columns' data types
    df = df.astype({
        "Date" : "datetime64[ns]",
        "Target_Currency" : "string",
        "Exchange_Rate" : "float64",
        "Rate_in_IDR" : "float64",
        "Base_Currency" : "string",
        "Description" : "string",
        "Created_at" : "datetime64[ns]"
    })

    #Reorder the columns
    column_order = ['Date','Target_Currency','Description','Exchange_Rate','Rate_in_IDR','Base_Currency','Created_at']
    df = df[column_order]

    #Return df 
    return df

In [9]:
idr_rate = extract_curr("IDR")

In [23]:
df = transform_curr(idr_rate)

df.head()

,Date,Target_Currency,Description,Exchange_Rate,Rate_in_IDR,Base_Currency,Created_at
0,2025-10-20,AED,United Arab Emirates Dirham,0.000222,4504.504505,IDR,2025-10-20 13:28:22
1,2025-10-20,AFN,Afghan Afghani,0.004020,248.756219,IDR,2025-10-20 13:28:22
2,2025-10-20,ALL,Albanian Lek,0.004997,200.120072,IDR,2025-10-20 13:28:22
3,2025-10-20,AMD,Armenian Dram,0.023214,43.077453,IDR,2025-10-20 13:28:22
4,2025-10-20,ANG,Netherlands Antillean Guilder,0.000108,9259.259259,IDR,2025-10-20 13:28:22


In [18]:
df[df['Target_Currency'] == 'JPY']

,Date,Target_Currency,Description,Exchange_Rate,Rate_in_IDR,Base_Currency,Created_at
73,2025-10-20,JPY,Japanese Yen,0.009089,110.023105,IDR,2025-10-20 13:11:20


In [20]:
def load_to_mysql(df):
  #connect the MySQL database
  usr = 'root'
  pwd = 'root'
  host = '127.0.0.1'
  port = 3306
  dbName = 'currency'
  tableName = 'exchange_rate'
  try:
    engine = create_engine(f"mysql+mysqldb://{usr}:{pwd}@{host}:{port}/{dbName}",
                           echo=True,
                           future=True)
    #Send the dataframes to MySQL table
    df.to_sql(name=tableName, con=engine, if_exists="append", index=False)

    print("Data berhasil ditambahkan ke tabel 'exchange_rate' di MySQL")
  except Exception as e:
    print("Gagal memuat data ke mysql:",e)

In [21]:
df = load_to_mysql(df)

2025-10-20 13:23:42,389 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2025-10-20 13:23:42,390 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 13:23:42,392 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2025-10-20 13:23:42,392 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 13:23:42,393 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2025-10-20 13:23:42,394 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 13:23:42,395 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-20 13:23:42,397 INFO sqlalchemy.engine.Engine DESCRIBE `currency`.`exchange_rate`
2025-10-20 13:23:42,397 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-20 13:23:42,405 INFO sqlalchemy.engine.Engine INSERT INTO exchange_rate (`Date`, `Target_Currency`, `Description`, `Exchange_Rate`, `Rate_in_IDR`, `Base_Currency`, `Created_at`) VALUES (%s, %s, %s, %s, %s, %s, %s)
2025-10-20 13:23:42,405 INFO sqlalchemy.engine.Engine [generated in 0.00160s] [(datetime.datetime(2025, 10, 20, 0, 0), 

In [24]:
if df.isnull().sum().any():
    print("⚠️ Warning: terdapat nilai kosong di dataset.")